In [6]:
import matplotlib.pyplot as plt
import numpy as np
from vpython import *
from sympy import *
def column(matrix, i):
    return [row[i] for row in matrix]

## My version

In [7]:
def removerlaps(project):
    #Remove overlaps
    #shift every z-plane point by z*0.1 in the y-direction, and subdivide every line segment into 10.
    l = len(project)
    #project = project.astype(float)
    for i in range(l):
        project[i][0] = project[i][0] + 0.1*project[i][2]
        project[i][1] = project[i][1] + 0.1*project[i][2]
    half = project
    new = project
    inc = 0
    for i in range(l-1):
        p = [project[i][0], project[i][1]]
        d3 = project[(i+1)] - project[i]
        d = [d3[0], d3[1]]
        add = np.zeros(shape=(9,3))
        if (d3[2]==0):
            for j in range(1,10):
                add[j-1] = [p[0] + 0.1*j*d[0], p[1] + 0.1*j*d[1], project[i][2]]
            new = np.concatenate((new[0: inc*9 + i +1], add, new[inc*9 +i+1:]), axis=0)
            inc +=1
    project = new
    return project

In [8]:
def alexpolynomial(x,y,z):
    points = np.zeros(shape=(len(x),3))
    points[:,0]=x
    points[:,1]=y
    points[:,2]=z
    points = removerlaps(points) #Should create a shear transformation in x- and y-axis to remove overlaps.
    #points = points*10

    #Remove extraneous 3D points
    deletes = []
    for i in range(0,len(points)-1):
        if (points[i][0]==points[i+1][0]) and (points[i][1]==points[i+1][1]):
            deletes.append(i+1)       
    project = np.delete(points, deletes, axis=0)

    #Finding the crossing points
    num = 0
    oindex = []
    uindex = []
    for i in range(1,len(project)-1):
        point1 = project[i]
        for j in range(i+1,len(project)):
            point2 = project[j]
            if (point1[0]==point2[0]) and (point1[1]==point2[1]):
                num = num + 1
                if (point1[2]>point2[2]):
                    oindex.append(i)
                    uindex.append(j)
                else:
                    oindex.append(j)
                    uindex.append(i)
                break

    project = np.delete(project, 2, axis=1)   #reduce to the string to a 2D projection             
    l = len(project)
    t = symbols('t')
    M = zeros(num, num) #Create empty matrix for alexander polynomial
    
    #Alterations: Sorting the lists of crossings, and deleting consecutive ones.
    over = np.array(oindex)
    under = np.array(uindex)
    over = over[np.argsort(under)] #Sorts overcrossings with their corresponding undercrossing (sorted below)
    under = np.sort(under)
    dels = np.array([], dtype=int)
    ll = len(under)
    for i in range(ll):
        a = under[i]
        b = under[(i+1)%ll]
        c = over[i]
        d = over[(i+1)%ll]
        
        if abs(a-b)==1 and abs(c-d)==1:
            dels = np.append(dels,int(i))
    
    under = np.delete(under, dels)
    over = np.delete(over, dels)
    oindex = over
    uindex = under
    num = len(over)

    #Iterate over the number of crossings
    for i in range(num):
        over = project[int((oindex[i]+1)%l)] - project[int(oindex[i])]
        under = project[int((uindex[i]+1)%l)] - project[int(uindex[i])]
        if np.cross(over,under) == -1:
            pol = [1-t, t, -1] #order is: [overstrand arc, understrand head arc, understrand tail arc]
        else:
            pol = [1-t, -1, t]

        endarc = [a-1 for a in uindex] #define the arc by the end of each arc,i.e. the point where it is cut off 
                                    #by the overstrand at the crossing. Then the 'name' of the arc is the index.

        #for overstrand arc
        list1 = endarc + [oindex[i]]
        list1 = sorted(list1)
        arc1 = list1.index(int(oindex[i]))
        if arc1 > (num-1):
            arc1 = 0
        M[i*(num) + arc1] = pol[0]

        #for understrand head arc
        list2 = endarc + [uindex[i]+1]
        list2 = sorted(list2)
        arc2 = list2.index(uindex[i]+1)
        if arc2 > (len(endarc)-1):
            arc2 = 0
        M[i*(num) + arc2] = pol[1]

        #for understrand tail arc
#         list3 = endarc + [uindex[i]-1]
#         list3 = sorted(list3)
        list3 = sorted(endarc)
        arc3 = list3.index(uindex[i]-1)
        M[i*(num) + arc3] = pol[2]

    if (num==0) or (num==1):
        M = Matrix([1])
    Mdupe = 1*M
    M.col_del(0)
    M.row_del(0)
    return M.det()

## DP's version

In [21]:
def d2(x,y,z):
    ''' Creates a modified projection of the knot in the X and Y planes '''
    '''Scales up the full knot by a certain amount (which depends on the max no. of spaces a point will end up being moved)
    Moves every point up and right depending on its z-coordinate
    Adds integer lattice points between the scaled up horizontal and vertical lines'''
    
    l = len(x)
    spread = np.amax(z) - np.amin(z)
    x = x*int(spread+1)
    y = y*int(spread+1)
    
    x += z
    y += z
    
    j = 0
    base = np.arange(1,spread+1,1)
    base2 = np.ones(int(spread))
    for i in range(l):
        try:
            if x[j] == x[j+1]:
                x = np.insert(x, j+1, x[j]*base2)
                y = np.insert(y, j+1, y[j]+(base*np.sign(y[j+1]-y[j])))
                z = np.insert(z, j+1, z[j]*base2)
                j += int(spread+1)
            else:
                if y[j] == y[j+1]:
                    x = np.insert(x, j+1, x[j]+(base*np.sign(x[j+1]-x[j])))
                    y = np.insert(y, j+1, y[j]*base2)
                    z = np.insert(z, j+1, z[j]*base2)
                    j += int(spread+1)
                else:
                    j += 1
        except IndexError:
            #print('ERROR!')
            break
    
    return x,y,z

def crossings(proj, z):
    ''' Returns an ordered list of the undercrossings, and their corresponding overcrossings '''
    
    over = np.array([])
    under = np.array([])
    for i in range(1,len(proj)):
        for j in range(i+1,len(proj)): #Check for repetition only in the points after so the same crossing isn't checked twice
            if all(proj[i] == proj[j]):
                front = [i,j][np.argmax([z[i],z[j]])]
                back = [i,j][np.argmin([z[i],z[j]])]
                
                over = np.append(over, front)
                under = np.append(under, back)
                break
    
    over = over[np.argsort(under)] #Sorts overcrossings with their corresponding undercrossing (sorted below)
    under = np.sort(under) #Sorts undercrossings in order of first to last
    return under, over

def alexpolynomial(x,y,z):
    """ Assumptions that must hold true: """
    ''' Every point of the knot lies on an integer 3D coordinate '''
    ''' You have the x,y,z coordinates for the curve '''
    """ Assumptions that if violated, should not break the function: """
    ''' No two consecutive undercrossings are also consecutive points in the projection '''
    ''' There are no overlapping line segments in the projection '''
    
    x_,y_,z_ = d2(x,y,z)
    proj = np.array([x_,y_]).T
    us, os = crossings(proj, z_)
    ulen = len(us)
    ubins = np.array([0] + us.tolist() + [len(proj)-1])
    ubins2 = us
    #Demarcates the arcs into bins with the endpoints being the undercrossings
    #The undercrossing itself is placed into the bin (arc) it is the left edge of
    
    l = len(proj)
    
    t = symbols('t')
    alexvars = [1, t, -t, -1]
    alexmat = zeros(ulen)
    
    for i in range(ulen):        
        vec1 = proj[int(os[i]+1)%(l-1)] - proj[int(os[i])]
        vec2 = proj[int(us[i]+1)%(l-1)] - proj[int(us[i])]
        if np.cross(vec1,vec2) == -1: #Point after overcrossing --> Point after undercrossing : Clockwise
            ps = [os[i], us[i]+0.1, os[i], us[i]-0.1]
        else:  #Point after overcrossing --> Point after undercrossing : Anti-clockwise
            ps = [os[i], us[i]-0.1, os[i], us[i]+0.1]
        
        for j in range(4):
            arcbin,_ = np.histogram(ps[j], bins=ubins)
            arc1 = np.argmax(arcbin)%ulen #Which arc is the point in, with the first and last bin being the same arc
            alexmat[ulen*i + arc1] += alexvars[j]
    
    try:
        alexmat.col_del(0)
        alexmat.row_del(0)
        al = alexmat.det()
        while rem(al, t) == 0:
            al = cancel(al/t)
        if rem(al, t) < 0:
            al *= -1
        return al
    except IndexError:
        return 1

In [22]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Trefoil.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))

t**2 - t + 1

In [23]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Figure 8.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))
# display(alexpolynomial(y,z,x)[0])
# display(alexpolynomial(z,x,y)[0])

t**2 - 3*t + 1

In [25]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Not Knots - Sheet1.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))
# display(alexpolynomial(y,z,x)[0])
# display(alexpolynomial(z,x,y)[0])

t**2 - t + 1

In [26]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/fancy figure 8.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))

t**2 - 3*t + 1

In [29]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/cinquefoil knot.csv', delimiter=',', unpack=True)
display(simplify(alexpolynomial(x,y,z)/-symbols('t')**2))

-t**2 + t - 1 + 1/t - 1/t**2

In [31]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/3 twist knot.csv', delimiter=',', unpack=True)
display(simplify(alexpolynomial(x,y,z)/symbols('t')**2))

2 - 3/t + 2/t**2

In [33]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/3 twist knot alt.csv', delimiter=',', unpack=True)
display(simplify(alexpolynomial(x,y,z)/symbols('t')**2))

2 - 3/t + 2/t**2

In [35]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/unknot.csv', delimiter=',', unpack=True)
alexpolynomial(x,y,z)

1

In [36]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/unknot variant.csv', delimiter=',', unpack=True)
alexpolynomial(x,y,z)

1

In [37]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Alex of 0.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))

1

In [38]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/3 crossing unknot.csv', delimiter=',', unpack=True)
display(alexpolynomial(x,y,z))

1

In [39]:
x = [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0]
y = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
z = [0.0, 0.0, 0.0, -1.0, -1.0, -2.0, -2.0, -2.0, -2.0, -3.0, -3.0, -3.0, -2.0, -1.0, -1.0, 0.0, 0.0]

display(alexpolynomial(x,y,z))

IndexError: index 70 is out of bounds for axis 0 with size 68